In [1]:
import warnings
warnings.filterwarnings(action='ignore')

import random
import pandas as pd
import numpy as np

SEED = 43
random.seed(SEED)
np.random.seed(SEED)

In [2]:
from dataset import get_data

In [3]:
DATA_PATH = "../data/kaggle_data/"

In [4]:
# make dataset

def data_extract(df_dict, df, name, cols):
       for col in cols:
              df_dict[f"{name}_{col}"] = df.loc[df[col] != 0, col]
       return df_dict

train_data_list = []
val_data_list = []

pmj_df = pd.read_csv(DATA_PATH + "pjm_elec_daily.csv")
pmj_cols = ['AEP', 'COMED', 'DAYTON', 'DEOK', 'DOM', 'DUQ', 'FE', 'NI', 'PJME', 'PJMW'] # 'PJM_Load', 'EKPC'

df_dict = {}

df_dict = data_extract(df_dict, pmj_df, 'PMJ', pmj_cols)

for df_name in df_dict:
    df = df_dict[df_name]
    train_data, val_data, scaler = get_data(df)
    print(df_name, len(train_data), len(val_data))
    train_data_list.append((train_data, df_name, scaler))
    val_data_list.append((val_data, df_name, scaler))

PMJ_AEP 4044 1011
PMJ_COMED 2217 555
PMJ_DAYTON 4044 1011
PMJ_DEOK 1925 482
PMJ_DOM 3874 969
PMJ_DUQ 3970 993
PMJ_FE 2096 525
PMJ_NI 1949 488
PMJ_PJME 4847 1212
PMJ_PJMW 4775 1194


In [5]:
tmp_data, t, scaler = train_data_list[0]

In [6]:
tmp_data = pd.DataFrame(tmp_data)

In [7]:
from util import plot_ac
diff = [7]
for tmp_data, t, scaler in train_data_list[:0]:
    tmp_data = pd.DataFrame(tmp_data)
    plot_ac(tmp_data, t, diff)

In [14]:
from statsmodels.tsa.arima.model import ARIMA

model = ARIMA(tmp_data, order=(0,1,1))
model_fit = model.fit()
print(model_fit.summary())

                               SARIMAX Results                                
Dep. Variable:                      0   No. Observations:                 4044
Model:                 ARIMA(0, 1, 1)   Log Likelihood                6514.642
Date:                Thu, 02 May 2024   AIC                         -13025.285
Time:                        15:52:28   BIC                         -13012.675
Sample:                             0   HQIC                        -13020.817
                               - 4044                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ma.L1          0.3444      0.016     21.298      0.000       0.313       0.376
sigma2         0.0023   4.57e-05     51.030      0.000       0.002       0.002
Ljung-Box (L1) (Q):                  32.30   Jarque-

In [24]:
forecast = model_fit.forecast(steps=100)

In [25]:
forecast

4044    0.595967
4045    0.595967
4046    0.595967
4047    0.595967
4048    0.595967
          ...   
4139    0.595967
4140    0.595967
4141    0.595967
4142    0.595967
4143    0.595967
Name: predicted_mean, Length: 100, dtype: float64